In [3]:
#!pip install python-igraph
import igraph
from igraph import Graph, EdgeSeq

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import copy

dataset = pd.read_csv(r'H:\RUET\3--2\CSE 3210- Sessional on 3209\Decision_Tree\Book.csv')

X = dataset.iloc[:,0:].values

total_rows=len(dataset.axes[0]) 
total_cols=len(dataset.axes[1])
str1=dataset.columns[total_cols-1]
#print(str1)
uniq=np.unique(dataset[str1])
#print(uniq)
print(total_rows,total_cols)
print(X)


class Node(object):
    def __init__(self):
        self.value = None
        self.decision = None
        self.childs = None

14 5
[['<=30' 'High' 'No' 'Fair' 'No']
 ['<=30' 'High' 'No' 'Excellent' 'No']
 ['31-40' 'High' 'No' 'Fair' 'Yes']
 ['>40' 'Medium' 'No' 'Fair' 'Yes']
 ['>40' 'Low' 'Yes' 'Fair' 'Yes']
 ['>40' 'Low' 'Yes' 'Excellent' 'No']
 ['31-40' 'Low' 'Yes' 'Excellent' 'Yes']
 ['<=30' 'Medium' 'No' 'Fair' 'No']
 ['<=30' 'Low' 'Yes' 'Fair' 'Yes']
 ['>40' 'Medium' 'Yes' 'Fair' 'Yes']
 ['<=30' 'Medium' 'Yes' 'Excellent' 'Yes']
 ['31-40' 'Medium' 'No' 'Excellent' 'Yes']
 ['31-40' 'High' 'Yes' 'Fair' 'Yes']
 ['>40' 'Medium' 'No' 'Excellent' 'No']]


In [2]:
def findEntropy(data, rows):
    class1 = 0.0
    class2 = 0.0
    ans = -1
    idx = len(data[0]) -1
    entropy = 0
    
    for i in rows:
        if data[i][idx] == uniq[1]:
            class1 += 1
        else:
            class2 += 1

    x = class1/(class1+class2)
    y = class2/(class1+class2)
    #print(class1)
    #print(class2)
    if x != 0 and y != 0:
        entropy = -1 * (x*math.log2(x) + y*math.log2(y))
    if x == 1:
        ans = 1
    if y == 1:
        ans = 0
    return entropy, ans

rows = [i for i in range(0, total_rows)]
val=findEntropy(X,rows)
m_str="Entropy is: "

print(m_str)
print(val)

Entropy is: 
(0.9402859586706311, -1)


In [5]:
def findMaxGain(data, rows, columns):
    maxGain = 0
    retidx = -1
    entropy, ans = findEntropy(data, rows)
    
    if entropy == 0:
        return maxGain, retidx, ans

    for j in columns:
        mydict = {}
        idx = j
        for i in rows:
            key = data[i][idx]
            if key not in mydict:
                mydict[key] = 1
            else:
                mydict[key] = mydict[key] + 1
        gain = entropy

        #print(mydict)
        for key in mydict:
            yes = 0
            no = 0
            for k in rows:
                if data[k][j] == key:
                    if data[k][-1] == uniq[1]:
                        yes = yes + 1
                    else:
                        no = no + 1
            #print(key,yes, no)
            x = yes/(yes+no)
            y = no/(yes+no)
            # print(x, y)
            if x != 0 and y != 0:
                gain += (mydict[key] * (x*math.log2(x) + y*math.log2(y)))/total_rows
        #print(gain)
        if gain > maxGain:
            maxGain = gain
            retidx = j
    return maxGain, retidx, ans

rows = [i for i in range(0, total_rows)]
columns = [i for i in range(0, total_cols-1)]
str1="Maximum Information Gain: "
print(str1)
val1=findMaxGain(X, rows, columns)
print(val1)
str1="Root is: "
val2=val1[1]
print(str1)
print(dataset.columns[val2])

Maximum Information Gain: 
(0.24674981977443927, 0, -1)
Root is: 
Age


In [6]:
def buildTree(data, rows, columns):

    maxGain, idx, ans = findMaxGain(X, rows, columns)

    root = Node()
    root.childs = []
    
    if maxGain == 0:
        if ans == 1:
            root.value = uniq[1]
        else:
            root.value = uniq[0]
        return root

    root.value = dataset.columns[idx]
    mydict = {}
    for i in rows:
        key = data[i][idx]
        if key not in mydict:
            mydict[key] = 1
        else:
            mydict[key] += 1

    newcolumns = copy.deepcopy(columns)
    newcolumns.remove(idx)
    for key in mydict:
        newrows = []
        for i in rows:
            if data[i][idx] == key:
                newrows.append(i)
        # print(newrows)
        temp = buildTree(data, newrows, newcolumns)
        temp.decision = key
        root.childs.append(temp)
    return root

def traverse(root,dep):
    for i in range(dep):
        print(" ",end=' ')
    print(root.decision,end=' ')
    print('->',end=' ')
    print(root.value)
    #print(root.value)
    dep+=4
    n = len(root.childs)
    if n > 0:
        for i in range(0, n):
            traverse(root.childs[i],dep)
            
def calculate():
    rows = [i for i in range(0, total_rows)]
    columns = [i for i in range(0, total_cols-1)]
    root = buildTree(X, rows, columns)
    root.decision = 'ROOT'
    dep=1
    traverse(root,dep)


calculate()

  ROOT -> Age
          <=30 -> Student
                  No -> No
                  Yes -> Yes
          31-40 -> Yes
          >40 -> Credit_rating
                  Fair -> Yes
                  Excellent -> No
